# ISO 14053:2021(E) 環境管理 - 物質流成本會計(MFCA)

本專案成果展示了一套使用 C# 語言開發的程式庫，其核心目的是協助使用者理解並實作 ISO 14053:2021(E) 的方法論。為了使這套複雜的標準更易於吸收與應用，我進一步利用 Jupyter Notebook 撰寫了互動式的介紹文件與可執行的程式範例，將理論說明與程式實踐緊密結合，提供了一套完整的學習與實作指南。

作者 周敬斐 (Mahler Chou) 

### 首次使用必須先引入 MfcaLib.cs 的函示庫

In [10]:
#load "MfcaLibs.cs"

### 建立一個製程站點物件 (Process)
根據 ISO 14053 的說明，原本稱為量化中心 (Quantity Center) 為了更貼切大眾的使用方式，更名為製程 (Process)。以下的C#範例程式碼是宣告一個製程物件 p1 的變數，隨後新增物料、廢棄物處理成本等等都是以 p 作為該製程的範圍。換句話說，未來如果有多個製程，就可以用 a1, a2, a3 自由命名。

宣告一個 Process 變數依序要填入製程名稱、開工日期、作業時間與預計完成數量，這些基本資料會用於最後的報表輸出上。

In [11]:
Process p1 = new("Cutting Department", "2025/10/1", "30 days", 2000);

### 輸入該製程站使用的物料
我們假設這一個製程是鋼板的切割部門，其投入的主要原料是不同尺寸厚度的鋼板，單價與數量彼此各異。

| 物料名稱 | 單價 | 投入數量 | 成本  | 生產耗用數量 | 不良損耗數量 |
| :---      | ---:       | ---:    | ---: | ---:            | ---:         |   
| Steel sheet A|13|1207|15691|896|311|
| Steel sheet B|8|600|4800|223|377 |
| Steel sheet C|30|120|3600|88|32 |
| Steel sheet D|20|602|12040|455|147|
| Steel sheet E|36|180|6480|135|45|

為了將上述的資料輸入到 p1 當中，我們將使用 Process 提供的函數方法 
> `SetTargetMaterial(物料名稱, 單價, 耗用量, 損耗量)`

In [12]:
p1.SetTargetMaterial("Steel sheet A", 13, 896, 311);
p1.SetTargetMaterial("Steel sheet B", 8, 223, 377);
p1.SetTargetMaterial("Steel sheet C", 30, 88, 32);
p1.SetTargetMaterial("Steel sheet D", 20, 455, 147);
p1.SetTargetMaterial("Steel sheet E", 36, 135, 45);

### 顯示物料清單
再將這個製程站所耗用的物料都輸入到物件 p1 之後，可以呼叫 `p1.ShowMaterial()` 將目前製程物件內的資料內容印出。

> _請注意在輸出結果中會顯示物料的分配比例 (Allocation Ratio)，這個比例會用來分配能源成本與系統成本。_

In [13]:
p1.ShowMaterial()

(TARGET MATERIALS)
Material Name        Unit Price Input      Cost       | Product    Cost       | Loss       Cost       | Loss Ratio 
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ---------- | ----------
Steel sheet A                13       1207      15691 |        896      11648 |        311       4043 |      25.8%
Steel sheet B                 8        600       4800 |        223       1784 |        377       3016 |      62.8%
Steel sheet C                30        120       3600 |         88       2640 |         32        960 |      26.7%
Steel sheet D                20        602      12040 |        455       9100 |        147       2940 |      24.4%
Steel sheet E                36        180       6480 |        135       4860 |         45       1620 |      25.0%
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ---------- | 
Subtotal of MTL.                      2709      42611 |       1797    

> _在生產的過程中，有些物料非主要原物料，仍會記錄到生產的投料清單當中。舉例來說，潤滑油就是一個例子，機器在切割鋼板的過程中，會需要使用潤滑油。由於非主要原料，或是對於成本影響比較小，該物料就指定為不參加正負產品比例的計算。這個計算的比例會影響到後續計算能源與系統成本。_

In [14]:
p1.SetTargetMaterial("Machine oil", 2, 0, 28, false);
p1.ShowMaterial();

(TARGET MATERIALS)
Material Name        Unit Price Input      Cost       | Product    Cost       | Loss       Cost       | Loss Ratio 
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ---------- | ----------
Steel sheet A                13       1207      15691 |        896      11648 |        311       4043 |      25.8%
Steel sheet B                 8        600       4800 |        223       1784 |        377       3016 |      62.8%
Steel sheet C                30        120       3600 |         88       2640 |         32        960 |      26.7%
Steel sheet D                20        602      12040 |        455       9100 |        147       2940 |      24.4%
Steel sheet E                36        180       6480 |        135       4860 |         45       1620 |      25.0%
* Machine oil                 2         28         56 |          0          0 |         28         56 |     100.0%
-------------------- ---------- ---------- ---------- | ----

### 廢棄物管理成本
添加廢棄物管理成本請使用函數 `SetWasteMgmtCost(廢棄物名稱, 成本)`，添加完畢後可以呼叫 `ShowWasteMgmt()` 印出廢棄物清單。

In [15]:
p1.SetWasteMgmtCost("Waste steel scrapts", 365);
p1.ShowWasteMgmt();

(WASTE MANAGEMENT)
Item Name                                             |                       |            Cost      
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------
Waste steel scrapts                                   |                       |            365       
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------
Subtotal                                              |                       |            365       
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------


### 能源成本
添加能源成本請使用 `SetEneryCost(能源項目名稱, 成本)`，呼叫 `ShowEnergyCost()` 可以印出目前製程的能源成本清單。

> _注意此次的能源成本分配是根據物料分配(正產出/負產出)比例來計算_

In [16]:
p1.SetEnergyCost("Electricity", 1660);
p1.ShowEnergyCost();

(ENERGY COSTS)
Item Name            Unit Price Quantity   Cost       | Ratio(+)   Cost       | Ratio(-)   Cost      
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------
Electricity                                      1660 |      66.3%       1101 |      33.7%        559
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------
SUBTOTAL                                        1,660 |                 1,101 |                   559
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------


### 系統成本
那些設備攤提、機器運轉成本、人力等等額外成本都計入在系統成本當中。若要添加系統成本，請使用 SetSystemCost 函數，若要查看製程 p1 內的系統成本項目則是使用 ShowSystemCost 函數。

> _注意此次的系統成本分配也是根據物料分配(正產出/負產出)比例來計算_

In [17]:
p1.SetSystemCost("Labour", 45000);
p1.SetSystemCost("Depreciation", 58000);
p1.SetSystemCost("Other", 15000);
p1.ShowSystemCost();

(SYSTEM COSTS)
Item Name            Unit Price Quantity   Cost       | Ratio(+)   Cost       | Ratio(-)   Cost      
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------
Labour                                          45000 |      66.3%      29835 |      33.7%      15165
Depreciation                                    58000 |      66.3%      38454 |      33.7%      19546
Other                                           15000 |      66.3%       9945 |      33.7%       5055
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------
SUBTOTAL                                      118,000 |      66.3%     78,234 |      33.7%     39,766
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ----------


## 物質流成本會計(MFCA)計算總表

在這個範例設計的情境當中，製程 p1 是一個鋼鐵切割的工序。當資料輸入完畢之後，可以呼叫 Report 方法印出計算總表。這個計算總表與其情境是參考 ISO 14053:2021(E) 表 A.5 所建立，其計算結果也一致。

> 從這個計算總表來看，在這個製程約的財務損失為 $55,325 ，相較於全部成本為 32.8% 。 由此可以看出，這個製程站點還有許多可以改進生產的地方：像是 'Steel sheet B` 的廢料比例高達 62.8%。

透過以上的程式碼與使用說明，可以很快地計算每一個製程站點的損耗。這個計算結果還可以被再進一步處理，搭配工單、途程、BOM表，或是使用自動化程序自 ERP / MES 建立定期的追蹤稽核項目，藉此發揮 MFCA 以物質流動找出成本損失的精神。

In [18]:
p1.Report()

Selected production process: Cutting Department
Production period: 2025/10/1 - 30 days
Planned production volumn: 2000
(TARGET MATERIALS)
Material Name        Unit Price Input      Cost       | Product    Cost       | Loss       Cost       | Loss Ratio 
-------------------- ---------- ---------- ---------- | ---------- ---------- | ---------- ---------- | ----------
Steel sheet A                13       1207      15691 |        896      11648 |        311       4043 |      25.8%
Steel sheet B                 8        600       4800 |        223       1784 |        377       3016 |      62.8%
Steel sheet C                30        120       3600 |         88       2640 |         32        960 |      26.7%
Steel sheet D                20        602      12040 |        455       9100 |        147       2940 |      24.4%
Steel sheet E                36        180       6480 |        135       4860 |         45       1620 |      25.0%
* Machine oil                 2         28         56 | 